In [33]:
# data from http://grouplens.org/datasets/movielens/

In [34]:
import os
import numpy as np
#data_folder = os.path.join(os.path.expanduser("ml-100k\\")
data_folder = os.path.join(os.path.expanduser("~"), "Downloads\\fitchburg\\courses\\datamining\\")
print data_folder
ratings_filename = os.path.join(data_folder, "BX-Book-Ratings.csv")

C:\Users\swaroop\Downloads\fitchburg\courses\datamining\


In [35]:
import pandas as pd

In [36]:
#all_ratings = pd.read_csv(ratings_filename, delimiter=";", header=None, names = ["UserID", "MovieID", "Rating", "Datetime"])
all_ratings = pd.read_csv(ratings_filename, delimiter=";")
#all_ratings["Datetime"] = pd.to_datetime(all_ratings['Datetime'],unit='s')

all_ratings = all_ratings[np.isfinite(all_ratings['Book-Rating'])]
all_ratings['Book-Rating'] = all_ratings['Book-Rating'].apply(np.int64)

all_ratings[:5]

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [37]:
# As you can see, there are no review for most movies, such as #213
all_ratings[all_ratings["User-ID"] == 276729].sort("Book-Rating")  

C:\Users\swaroop\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,User-ID,ISBN,Book-Rating
3,276729,052165615X,3
4,276729,0521795028,6


In [38]:
# Not all reviews are favourable! Our goal is "other recommended books", so we only want favourable reviews
all_ratings["Favorable"] = all_ratings["Book-Rating"] > 3
all_ratings[10:15]

,User-ID,ISBN,Book-Rating,Favorable
10,276746,0425115801,0,False
11,276746,0449006522,0,False
12,276746,0553561618,0,False
13,276746,055356451X,0,False
14,276746,0786013990,0,False


In [39]:
all_ratings[all_ratings["User-ID"] == 276746][:5]

,User-ID,ISBN,Book-Rating,Favorable
10,276746,0425115801,0,False
11,276746,0449006522,0,False
12,276746,0553561618,0,False
13,276746,055356451X,0,False
14,276746,0786013990,0,False


In [40]:
# Sample the dataset. You can try increasing the size of the sample, but the run time will be considerably longer
#ratings = all_ratings[all_ratings['User-ID'].isin(range(20000))]  # & 

ratings = all_ratings[all_ratings["User-ID"].isin(range(200000))]

#ratings = all_ratings
print len(ratings)

817373


In [41]:
# We start by creating a dataset of each user's favourable reviews
favorable_ratings = ratings[ratings["Favorable"]]
favorable_ratings[:5]

print len(favorable_ratings)

310033


In [42]:
# We are only interested in the reviewers who have more than one review
favorable_reviews_by_users = dict((k, frozenset(v.values)) for k, v in favorable_ratings.groupby("User-ID")["ISBN"])
len(favorable_reviews_by_users)


54700

In [43]:
# Find out how many movies have favourable ratings
num_favorable_by_movie = ratings[["ISBN", "Favorable"]].groupby("ISBN").sum()
num_favorable_by_movie.sort("Favorable", ascending=False)[:5]

print len(num_favorable_by_movie)

print num_favorable_by_movie[:5]

278773
             Favorable
ISBN                  
 0330299891        1.0
 0375404120        0.0
 0586045007        0.0
 9022906116        1.0
 9032803328        0.0


C:\Users\swaroop\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [44]:
from collections import defaultdict

def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_support):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():
        for itemset in k_1_itemsets:
            #print type(itemset),itemset, reviews
            #break
            #break
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])

In [45]:
import sys
frequent_itemsets = {}  # itemsets are sorted by length
min_support = 50

# k=1 candidates are the isbns with more than min_support favourable reviews
frequent_itemsets[1] = dict((frozenset((ISBN,)), row["Favorable"])
                                for ISBN, row in num_favorable_by_movie.iterrows()
                                if row["Favorable"] > min_support)

print("There are {} books with more than {} favorable reviews".format(len(frequent_itemsets[1]), min_support))
sys.stdout.flush()
for k in range(2, 20):
    # Generate candidates of length k, using the frequent itemsets of length k-1
    # Only store the frequent itemsets
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k-1],
                                                   min_support)
    if len(cur_frequent_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k))
        #print(cur_frequent_itemsets)
        sys.stdout.flush()
        frequent_itemsets[k] = cur_frequent_itemsets
# We aren't interested in the itemsets of length 1, so remove those
del frequent_itemsets[1]

There are 274 books with more than 50 favorable reviews
I found 36 frequent itemsets of length 2
I found 13 frequent itemsets of length 3
I found 2 frequent itemsets of length 4
Did not find any frequent itemsets of length 5


In [46]:
print("Found a total of {0} frequent itemsets".format(sum(len(itemsets) for itemsets in frequent_itemsets.values())))

Found a total of 51 frequent itemsets


In [47]:
# Now we create the association rules. First, they are candidates until the confidence has been tested
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))
print("There are {} candidate rules".format(len(candidate_rules)))

There are 119 candidate rules


In [48]:
print(candidate_rules[:5])

[(frozenset(['0316666343']), '0385504209'), (frozenset(['0385504209']), '0316666343'), (frozenset(['0590353403']), '0439064864'), (frozenset(['0439064864']), '0590353403'), (frozenset(['0671001795']), '0312966970')]


In [49]:
# Now, we compute the confidence of each of these rules. This is very similar to what we did in chapter 1
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
rule_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules}


In [50]:
# Choose only rules above a minimum confidence level
min_confidence = 0.9

In [51]:
# Filter out the rules with poor confidence
rule_confidence = {rule: confidence for rule, confidence in rule_confidence.items() if confidence > min_confidence}
print(len(rule_confidence))

5


In [52]:
from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)

In [53]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise, conclusion))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends frozenset(['0439139597', '0590353403', '0439136350']) they will also recommend 0439064864
 - Confidence: 0.968

Rule #2
Rule: If a person recommends frozenset(['0590353403', '0439136350']) they will also recommend 0439064864
 - Confidence: 0.955

Rule #3
Rule: If a person recommends frozenset(['0439139597', '0590353403']) they will also recommend 0439064864
 - Confidence: 0.943

Rule #4
Rule: If a person recommends frozenset(['043935806X', '0439139597', '0439064864']) they will also recommend 0439136350
 - Confidence: 0.913

Rule #5
Rule: If a person recommends frozenset(['0439139597', '0439064864', '0590353403']) they will also recommend 0439136350
 - Confidence: 0.909



In [54]:
# Even better, we can get the movie titles themselves from the dataset
movie_name_filename = os.path.join(data_folder, "BX-Books.csv")
movie_name_data = pd.read_csv(movie_name_filename, delimiter=";", encoding = "latin-1", error_bad_lines = False, warn_bad_lines = False)
#movie_name_data.columns = ["MovieID", "Title", "Release Date", "Video Release", "IMDB", "<UNK>", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
print movie_name_data.shape
print movie_name_data.columns

(271360, 8)
Index([u'ISBN', u'Book-Title', u'Book-Author', u'Year-Of-Publication',
       u'Publisher', u'Image-URL-S', u'Image-URL-M', u'Image-URL-L'],
      dtype='object')


In [55]:
def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["ISBN"] == str(movie_id)]["Book-Title"]
    title = title_object.values[0]
    return title

In [56]:
get_movie_name('0425115801')

u'Lightning'

In [57]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    print premise
    print conclusion
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(str(conclusion))
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")


Rule #1
frozenset(['0439139597', '0590353403', '0439136350'])
0439064864
Rule: If a person recommends Harry Potter and the Goblet of Fire (Book 4), Harry Potter and the Sorcerer's Stone (Book 1), Harry Potter and the Prisoner of Azkaban (Book 3) they will also recommend Harry Potter and the Chamber of Secrets (Book 2)
 - Confidence: 0.968

Rule #2
frozenset(['0590353403', '0439136350'])
0439064864
Rule: If a person recommends Harry Potter and the Sorcerer's Stone (Book 1), Harry Potter and the Prisoner of Azkaban (Book 3) they will also recommend Harry Potter and the Chamber of Secrets (Book 2)
 - Confidence: 0.955

Rule #3
frozenset(['0439139597', '0590353403'])
0439064864
Rule: If a person recommends Harry Potter and the Goblet of Fire (Book 4), Harry Potter and the Sorcerer's Stone (Book 1) they will also recommend Harry Potter and the Chamber of Secrets (Book 2)
 - Confidence: 0.943

Rule #4
frozenset(['043935806X', '0439139597', '0439064864'])
0439136350
Rule: If a person recommen

In [58]:
# Evaluation using test data
test_dataset = all_ratings[~all_ratings['User-ID'].isin(range(200000))]
test_favorable = test_dataset[test_dataset["Favorable"]]
#test_not_favourable = test_dataset[~test_dataset["Favourable"]]
test_favorable_by_users = dict((k, frozenset(v.values)) for k, v in test_favorable.groupby("User-ID")["Book-Rating"])
#test_not_favourable_by_users = dict((k, frozenset(v.values)) for k, v in test_not_favourable.groupby("UserID")["MovieID"])
#test_users = test_dataset["UserID"].unique()

print len(test_favorable_by_users)

21628


In [59]:
test_dataset[:5]

,User-ID,ISBN,Book-Rating,Favorable
0,276725,034545104X,0,False
1,276726,0155061224,5,True
2,276727,0446520802,0,False
3,276729,052165615X,3,False
4,276729,0521795028,6,True


In [60]:
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)

for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

In [61]:
# Now, we compute the confidence of each of these rules. This is very similar to what we did in chapter 1
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
rule_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules}


In [62]:
test_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
                   for candidate_rule in rule_confidence}
print(len(test_confidence))

119


In [63]:
sorted_test_confidence = sorted(test_confidence.items(), key=itemgetter(1), reverse=True)
print(sorted_test_confidence[:5])

[((frozenset(['0439139597', '0590353403', '0439136350']), '0439064864'), 0.967741935483871), ((frozenset(['0590353403', '0439136350']), '0439064864'), 0.9545454545454546), ((frozenset(['0439139597', '0590353403']), '0439064864'), 0.9428571428571428), ((frozenset(['043935806X', '0439139597', '0439064864']), '0439136350'), 0.9130434782608695), ((frozenset(['0439139597', '0439064864', '0590353403']), '0439136350'), 0.9090909090909091)]


In [64]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Train Confidence: {0:.3f}".format(rule_confidence.get((premise, conclusion), -1)))
    print(" - Test Confidence: {0:.3f}".format(test_confidence.get((premise, conclusion), -1)))
    print("")

Rule #1
Rule: If a person recommends Harry Potter and the Goblet of Fire (Book 4), Harry Potter and the Sorcerer's Stone (Book 1), Harry Potter and the Prisoner of Azkaban (Book 3) they will also recommend Harry Potter and the Chamber of Secrets (Book 2)
 - Train Confidence: 0.968
 - Test Confidence: 0.968

Rule #2
Rule: If a person recommends Harry Potter and the Sorcerer's Stone (Book 1), Harry Potter and the Prisoner of Azkaban (Book 3) they will also recommend Harry Potter and the Chamber of Secrets (Book 2)
 - Train Confidence: 0.955
 - Test Confidence: 0.955

Rule #3
Rule: If a person recommends Harry Potter and the Goblet of Fire (Book 4), Harry Potter and the Sorcerer's Stone (Book 1) they will also recommend Harry Potter and the Chamber of Secrets (Book 2)
 - Train Confidence: 0.943
 - Test Confidence: 0.943

Rule #4
Rule: If a person recommends Harry Potter and the Order of the Phoenix (Book 5), Harry Potter and the Goblet of Fire (Book 4), Harry Potter and the Chamber of Sec